In [25]:


# Imports
import pandas as pd
import plotly.graph_objects as go
import ipywidgets as widgets
import plotly.io as pio
from IPython.display import display

# Force Plotly to render in Colab
pio.renderers.default = 'colab'

import plotly.io as pio
pio.renderers.default = 'colab'


In [81]:
# Load matches data
matches_df = pd.read_csv('matches.csv')
# Load deliveries dataset
deliveries_df = pd.read_csv('deliveries.csv')


# Standardize team names
team_mapping = {
    'Delhi Daredevils': 'Delhi Capitals',
    'Deccan Chargers': 'Sunrisers Hyderabad',
    'Kings XI Punjab': 'Punjab Kings',
    'Royal Challengers Bengaluru': 'Royal Challengers Bangalore',
    'Rising Pune Supergiant': 'Rising Pune Supergiants',
}
matches_df.replace({'team1': team_mapping, 'team2': team_mapping, 'winner': team_mapping}, inplace=True)


In [64]:
matches_df.head()

,id,season,city,date,match_type,player_of_match,venue,team1,team2,toss_winner,toss_decision,winner,result,result_margin,target_runs,target_overs,super_over,method,umpire1,umpire2
0,335982,2007/08,Bangalore,2008-04-18,League,BB McCullum,M Chinnaswamy Stadium,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,223.0,20.0,N,NaN,Asad Rauf,RE Koertzen
1,335983,2007/08,Chandigarh,2008-04-19,League,MEK Hussey,"Punjab Cricket Association Stadium, Mohali",Punjab Kings,Chennai Super Kings,Chennai Super Kings,bat,Chennai Super Kings,runs,33.0,241.0,20.0,N,NaN,MR Benson,SL Shastri
2,335984,2007/08,Delhi,2008-04-19,League,MF Maharoof,Feroz Shah Kotla,Delhi Capitals,Rajasthan Royals,Rajasthan Royals,bat,Delhi Capitals,wickets,9.0,130.0,20.0,N,NaN,Aleem Dar,GA Pratapkumar
3,335985,2007/08,Mumbai,2008-04-20,League,MV Boucher,Wankhede Stadium,Mumbai Indians,Royal Challengers Bangalore,Mumbai Indians,bat,Royal Challengers Bangalore,wickets,5.0,166.0,20.0,N,NaN,SJ Davis,DJ Harper
4,335986,2007/08,Kolkata,2008-04-20,League,DJ Hussey,Eden Gardens,Kolkata Knight Riders,Sunrisers Hyderabad,Deccan Chargers,bat,Kolkata Knight Riders,wickets,5.0,111.0,20.0,N,NaN,BF Bowden,K Hariharan


In [32]:
# Total matches played per team per season
team_played = pd.melt(matches_df, id_vars=['season'], value_vars=['team1', 'team2'],
                      var_name='home_away', value_name='team')
team_played = team_played.groupby(['season', 'team']).size().reset_index(name='matches_played')

# Total matches won per team per season
team_wins = matches_df.groupby(['season', 'winner']).size().reset_index(name='matches_won')
team_wins.rename(columns={'winner': 'team'}, inplace=True)

# Merge and calculate win ratio
win_ratio = pd.merge(team_played, team_wins, on=['season', 'team'], how='left')
win_ratio['matches_won'] = win_ratio['matches_won'].fillna(0)
win_ratio['win_ratio'] = (win_ratio['matches_won'] / win_ratio['matches_played']).round(2)


In [105]:
import pandas as pd
import plotly.graph_objects as go

# Use win_ratio DataFrame (assumed already computed)
teams = sorted(win_ratio['team'].unique())

# Clean season labels
win_ratio['season_clean'] = win_ratio['season'].str.extract(r'(\d{4})').astype(int)

# Build one trace per team — all hidden except default
fig = go.Figure()

for team in teams:
    team_data = win_ratio[win_ratio['team'] == team]
    fig.add_trace(go.Scatter(
        x=team_data['season_clean'],
        y=team_data['win_ratio'],
        mode='lines+markers',
        name=team,
        visible=(team == 'Mumbai Indians'),
        line=dict(width=4),
        marker=dict(size=8),
        hovertemplate='Season: %{x}<br>Win Ratio: %{y:.0%}<extra></extra>'
    ))


# Build dropdown buttons for team toggle
dropdown_buttons = [
    dict(label=team,
         method='update',
         args=[{'visible': [t == team for t in teams]},
               {'title': f'🏆 Win Ratio by Season: {team}'}])
    for team in teams
]

# Update layout with embedded dropdown
fig.update_layout(
    updatemenus=[dict(
        buttons=dropdown_buttons,
        direction="down",
        showactive=True,
        x=1.15,
        xanchor="left",
        y=1,
        yanchor="top"
    )],
    title="🏆 Win Ratio by Season: Mumbai Indians",
    xaxis_title="Season",
    yaxis_title="Win Ratio",
    yaxis=dict(tickformat=".0%", range=[0, 1]),
    xaxis=dict(type='category', tickangle=-45),
    font=dict(size=15),
    height=600,
    width=950
)

fig.show()


# -----------------------------------------------------------------------------------------------------------------------------------

## Win Percentage + Trophies Won

In [55]:
# Total matches played per team
team_played_total = pd.melt(matches_df, value_vars=['team1', 'team2'], value_name='team')
team_played_total = team_played_total.groupby('team').size().reset_index(name='total_matches')

# Total matches won
team_wins_total = matches_df.groupby('winner').size().reset_index(name='total_wins')
team_wins_total.rename(columns={'winner': 'team'}, inplace=True)

# Trophies (title wins) – Only final match winners
finals = matches_df.dropna(subset=['season'])  # just in case
final_match_per_season = finals.groupby('season').apply(lambda x: x.iloc[-1]).reset_index(drop=True)
trophies = final_match_per_season['winner'].value_counts().reset_index()
trophies.columns = ['team', 'trophies']

# Merge all
win_summary = pd.merge(team_played_total, team_wins_total, on='team', how='left')
win_summary = pd.merge(win_summary, trophies, on='team', how='left')
win_summary.fillna(0, inplace=True)

# Calculate win %
win_summary['win_pct'] = (win_summary['total_wins'] / win_summary['total_matches'] * 100).round(2)


<ipython-input-55-0ba531a5587b>:11: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



In [61]:
import plotly.graph_objects as go
import math
from plotly.subplots import make_subplots

# Step 1: Filter and sort
filtered = win_summary[win_summary['total_matches'] >= 50].copy()
filtered = filtered.sort_values(by='trophies', ascending=False).reset_index(drop=True)

# Step 2: Grid setup
cols = 3
rows = math.ceil(len(filtered) / cols)

fig = make_subplots(
    rows=rows, cols=cols,
    specs=[[{"type": "indicator"}]*cols for _ in range(rows)]
)

# Step 3: Add one gauge per team
for idx, row in filtered.iterrows():
    r = (idx // cols) + 1
    c = (idx % cols) + 1

    fig.add_trace(go.Indicator(
        mode="gauge+number",
        value=row['win_pct'],
        number={
            'suffix': f"%\n🏆 {int(row['trophies'])}",
            'font': {'size': 20}
        },
        title={'text': row['team'], 'font': {'size': 18}},  # Bigger team name
        gauge={
            'axis': {'range': [0, 100]},
            'bar': {'color': 'deepskyblue'},
            'bgcolor': 'lightgray',
            'steps': [
                {'range': [0, 50], 'color': "#ffcccc"},
                {'range': [50, 75], 'color': "#ffe066"},
                {'range': [75, 100], 'color': "#b7f0ad"},
            ],
        },
        domain={'row': r-1, 'column': c-1}
    ), row=r, col=c)

# Step 4: Layout polish
fig.update_layout(
    title_text="🏏 IPL Legacy Dashboard: Win Percentage & Trophies of Top Teams (2008–2024)",
    height=300 * rows,
    width=1100,
    font=dict(size=14),
    margin=dict(t=100),
    title_font=dict(size=24)
)


fig.show()


## TOSS IMPACT

In [77]:
import pandas as pd
import plotly.graph_objects as go

# Load + standardize team names
matches_df = pd.read_csv('matches.csv')
team_mapping = {
    'Delhi Daredevils': 'Delhi Capitals',
    'Deccan Chargers': 'Sunrisers Hyderabad',
    'Kings XI Punjab': 'Punjab Kings',
    'Royal Challengers Bengaluru': 'Royal Challengers Bangalore',
    'Rising Pune Supergiant': 'Rising Pune Supergiants',
}
matches_df.replace({'team1': team_mapping, 'team2': team_mapping,
                    'winner': team_mapping, 'toss_winner': team_mapping}, inplace=True)

# Process data
toss_won = matches_df['toss_winner'].value_counts().reset_index()
toss_won.columns = ['team', 'toss_wins']

toss_match_win = matches_df[matches_df['toss_winner'] == matches_df['winner']]['toss_winner'].value_counts().reset_index()
toss_match_win.columns = ['team', 'toss_and_match_wins']

df = pd.merge(toss_won, toss_match_win, on='team', how='left').fillna(0)
df['toss_and_match_losses'] = df['toss_wins'] - df['toss_and_match_wins']
df = df.sort_values(by='toss_wins', ascending=False).reset_index(drop=True)

# Stacked bar chart
fig = go.Figure()

# Bar: Toss → Match Wins
fig.add_trace(go.Bar(
    x=df['team'],
    y=df['toss_and_match_wins'],
    name='✅ Toss → Match Wins',
    marker_color='#80dfff',
    text=df['toss_and_match_wins'],
    textposition='outside',
    hovertemplate='Toss Wins Converted: %{y}<br>Team: %{x}<extra></extra>'
))

# Bar: Toss → Match Losses
fig.add_trace(go.Bar(
    x=df['team'],
    y=df['toss_and_match_losses'],
    name='❌ Toss → Match Losses',
    marker_color='#1e3f66',
    text=df['toss_and_match_losses'],
    textposition='outside',
    hovertemplate='Toss Wins Not Converted: %{y}<br>Team: %{x}<extra></extra>'
))

# Layout beautification
fig.update_layout(
    barmode='stack',
    title='🎯 Toss-to-Match Conversion Breakdown (Polished View)',
    xaxis=dict(title='Team', tickangle=-40, tickfont=dict(size=12)),
    yaxis=dict(title='Total Toss Wins', gridcolor='lightgray'),
    font=dict(family='Arial', size=14),
    height=620,
    width=1100,
    plot_bgcolor='white',
    paper_bgcolor='white',
    legend=dict(x=1.02, y=1, bgcolor='rgba(255,255,255,0)', bordercolor='gray'),
    margin=dict(t=80, b=100)
)

fig.update_traces(marker_line_width=0.5)

fig.show()


## VENUES AND SCORED RUNS

In [96]:
stadium_locations = [
    {"venue": "Wankhede Stadium", "city": "Mumbai", "lat": 18.9388, "lon": 72.8258},
    {"venue": "M. Chinnaswamy Stadium", "city": "Bangalore", "lat": 12.9788, "lon": 77.5996},
    {"venue": "Eden Gardens", "city": "Kolkata", "lat": 22.5646, "lon": 88.3433},
    {"venue": "Narendra Modi Stadium", "city": "Ahmedabad", "lat": 23.0918, "lon": 72.5977},
    {"venue": "Arun Jaitley Stadium", "city": "Delhi", "lat": 28.6229, "lon": 77.2430},
    {"venue": "Rajiv Gandhi Intl. Stadium", "city": "Hyderabad", "lat": 17.4062, "lon": 78.5506},
    {"venue": "MA Chidambaram Stadium", "city": "Chennai", "lat": 13.0624, "lon": 80.2791},
    {"venue": "Sawai Mansingh Stadium", "city": "Jaipur", "lat": 26.8945, "lon": 75.8039}
]


In [104]:
import pandas as pd
import plotly.express as px

# Load datasets
matches_df = pd.read_csv('matches.csv')
deliveries_df = pd.read_csv('deliveries.csv')

# Standardize team names
team_mapping = {
    'Delhi Daredevils': 'Delhi Capitals',
    'Deccan Chargers': 'Sunrisers Hyderabad',
    'Kings XI Punjab': 'Punjab Kings',
    'Royal Challengers Bengaluru': 'Royal Challengers Bangalore',
    'Rising Pune Supergiant': 'Rising Pune Supergiants',
}
matches_df.replace({'team1': team_mapping, 'team2': team_mapping, 'winner': team_mapping}, inplace=True)

# Merge venue into deliveries
merged_df = deliveries_df.merge(matches_df[['id', 'venue']], left_on='match_id', right_on='id')

# Total runs scored per venue
venue_runs = merged_df.groupby('venue')['total_runs'].sum().reset_index()

# Matches played per venue
venue_matches = matches_df['venue'].value_counts().reset_index()
venue_matches.columns = ['venue', 'matches_played']

# Combine run + match data
venue_stats = pd.merge(venue_runs, venue_matches, on='venue')
venue_stats['avg_runs_per_match'] = (venue_stats['total_runs'] / venue_stats['matches_played']).round(2)

# Manually assign coordinates (expand this as needed)
stadium_coords = {
    'Wankhede Stadium': (18.9388, 72.8258),
    'M. Chinnaswamy Stadium': (12.9788, 77.5996),
    'Eden Gardens': (22.5646, 88.3433),
    'Narendra Modi Stadium': (23.0918, 72.5977),
    'Arun Jaitley Stadium': (28.6229, 77.2430),
    'Rajiv Gandhi Intl. Stadium': (17.4062, 78.5506),
    'MA Chidambaram Stadium': (13.0624, 80.2791),
    'Sawai Mansingh Stadium': (26.8945, 75.8039),
    'Himachal Pradesh Cricket Association Stadium': (32.1976, 76.3254),
    'Dr DY Patil Sports Academy': (19.0330, 73.0297),
    'Punjab Cricket Association Stadium': (30.7036, 76.7183),
    'Green Park': (26.4725, 80.3467),
    'Sheikh Zayed Stadium': (24.4672, 54.3717),
    'Sharjah Cricket Stadium': (25.3187, 55.4211),
    'Dubai International Cricket Stadium': (25.0458, 55.2319)
}

# Map coordinates to venue_stats
venue_stats['lat'] = venue_stats['venue'].map(lambda x: stadium_coords.get(x, (None, None))[0])
venue_stats['lon'] = venue_stats['venue'].map(lambda x: stadium_coords.get(x, (None, None))[1])

# Filter out venues with missing coords
venue_stats = venue_stats.dropna(subset=['lat', 'lon'])

# Plot using Plotly
fig = px.scatter_mapbox(
    venue_stats,
    lat='lat',
    lon='lon',
    hover_name='venue',
    hover_data={'total_runs': True, 'matches_played': True, 'avg_runs_per_match': True},
    size='total_runs',
    color='avg_runs_per_match',
    color_continuous_scale='Plasma',
    size_max=40,
    zoom=4.2,
    height=680,
    title='📍 All IPL Stadiums — Total Runs & Avg Runs per Match'
)

fig.update_layout(
    mapbox_style='carto-positron',
    margin={"r": 0, "t": 50, "l": 0, "b": 0},
    font=dict(size=14)
)

fig.show()
